In [8]:
from keras.layers import Dense, BatchNormalization, Activation, Dropout, InputLayer,Conv2D, MaxPool2D, Flatten
from keras.models import Sequential
from keras_diagram import ascii
from keras.models import Model
import keras.backend as K
import numpy as np

import random

from tqdm import tqdm
from tensorflow.contrib.data import TFRecordDataset
import tensorflow as tf


In [1]:
def baseline_cnn(shape, num_speakers=10, dropout=0.5):
    model = Sequential([
        InputLayer(input_shape=shape),

        Conv2D(filters=16, kernel_size=(3, 3), padding='same'),
        BatchNormalization(),
        Activation(activation='relu'),
        MaxPool2D(pool_size=(2, 2)),
        Dropout(0.5),

        Conv2D(filters=32, kernel_size=(3, 3), padding='same'),
        BatchNormalization(),
        Activation(activation='relu'),
        MaxPool2D(pool_size=(2, 2)),
        Dropout(0.5),

        Conv2D(filters=64, kernel_size=(3, 3), padding='same'),
        BatchNormalization(),
        Activation(activation='relu'),
        MaxPool2D(pool_size=(2, 2)),
        Dropout(0.5),
        Flatten(),

        Dense(512),
        BatchNormalization(),
        Activation(activation='relu'),
        Dropout(0.5),

        Dense(num_speakers, activation='softmax')
    ])
    print(ascii(model))
    return model

In [3]:
model = baseline_cnn((64, 64, 1), 46)

         InputLayer (None, 64, 64, 1)  
             Conv2D (None, 64, 64, 16) 
 BatchNormalization (None, 64, 64, 16) 
               Relu (None, 64, 64, 16) 
       MaxPooling2D (None, 32, 32, 16) 
            Dropout (None, 32, 32, 16) 
             Conv2D (None, 32, 32, 32) 
 BatchNormalization (None, 32, 32, 32) 
               Relu (None, 32, 32, 32) 
       MaxPooling2D (None, 16, 16, 32) 
            Dropout (None, 16, 16, 32) 
             Conv2D (None, 16, 16, 64) 
 BatchNormalization (None, 16, 16, 64) 
               Relu (None, 16, 16, 64) 
       MaxPooling2D (None, 8, 8, 64)   
            Dropout (None, 8, 8, 64)   
            Flatten (None, 4096)       
              Dense (None, 512)        
 BatchNormalization (None, 512)        
               Relu (None, 512)        
            Dropout (None, 512)        
              Dense (None, 46)         



In [6]:
model.load_weights("/home/jugs/Desktop/model.h5")

In [7]:
config = model.get_config()
weight = model.get_weights()

In [14]:
new_model = Model.from_config(config)
new_model.set_weights(weights)

TypeError: list indices must be integers or slices, not str

In [11]:
from tensorflow.python.saved_model import builder as saved_model_builder
from tensorflow.python.saved_model import utils
from tensorflow.python.saved_model import tag_constants, signature_constants
from tensorflow.python.saved_model.signature_def_utils_impl import     build_signature_def, predict_signature_def
from tensorflow.contrib.session_bundle import exporter


In [13]:
export_path = "/home/jugs/Desktop/logs/tf_servings_export"
builder = saved_model_builder.SavedModelBuilder(export_path)
signature = predict_signature_def(inputs={'images': new_model.input},
                                  outputs={'scores': new_model.output})

with K.get_session() as sess:
    builder.add_meta_graph_and_variables(sess=sess,
                                         tags=[tag_constants.SERVING],
                                         signature_def_map={'predict': signature})
    builder.save()

NameError: name 'new_model' is not defined